In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/stepik/m3. unsupervised/w4. topic modeling/4.4.2 postnauka'
!ls
!pip install bigartm

/content/drive/MyDrive/Colab Notebooks/stepik/m3. unsupervised/w4. topic modeling/4.4.2 postnauka
 batches
 bigartm.d4e5bd8384a4.invalid-user.log.INFO.20210515-035334.57
 bigartm.d4e5bd8384a4.invalid-user.log.WARNING.20210515-035335.57
 bigartm.INFO
 bigartm.WARNING
 compare.ipynb
 lectures_batches
 lectures.txt
 postnauka.ipynb
'postnauka - original.ipynb'
 PostnaukaPeerReview.ipynb
'postnauka template.ipynb'
     |████████████████████████████████| 1.9MB 7.3MB/s 


In [3]:
import artm
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("whitegrid", {'axes.grid' : False})
import numpy as np
import pandas as pd
import random
from sklearn.externals import joblib
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [5]:
batch_vectorizer = artm.BatchVectorizer(data_path='lectures.txt', data_format='vowpal_wabbit',
                                        target_folder='lectures_batches', batch_size=250)
T = 30   # количество тем
topic_names=["sbj"+str(i) for i in range(T-1)]+["bcg"]
model = artm.ARTM(num_topics=T, topic_names=topic_names, num_processors=2, class_ids={'text':1, 'author':1},
                  reuse_theta=True, cache_theta=True)
np.random.seed(1)
dictionary = artm.Dictionary('dict')
dictionary.gather(batch_vectorizer.data_path)
model.initialize(dictionary=dictionary)
model.scores.add(artm.TopTokensScore(name='top_tokens_score_mod1', class_id='text', num_tokens=15))
model.scores.add(artm.TopTokensScore(name='top_tokens_score_mod2', class_id='author', num_tokens=15))

model.regularizers.add(artm.SmoothSparsePhiRegularizer(tau=1e5, class_ids='text', dictionary='dict', topic_names='bcg'))
model.num_document_passes = 1
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=30)

topic_names_cleared = list(topic_names).remove('bcg')
model.regularizers.add(artm.SmoothSparsePhiRegularizer(tau=-1e5, class_ids='text', dictionary='dict',
                                                       topic_names=topic_names_cleared))
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)
tokens = model.score_tracker['top_tokens_score_mod1'].last_tokens
for topic_name in model.topic_names:
    print(topic_name, ': ', ' '.join(tokens[topic_name]))

sbj0 :  церковь власть средневековый император русь святой римский папа князь сага король александр византийский вера рим
sbj1 :  планета земля атмосфера солнце солнечный белый газ образовываться карлик вокруг вода поверхность спутник орбита океан
sbj2 :  система трудность поведение частота двигаться точность искусственный движение относительность тест час самка задание переход часы
sbj3 :  культура понятие культурный восприятие сознание феномен личность воспринимать ощущение чувство повседневный переживание детство этика индивид
sbj4 :  право литература искусство литературный юрист писатель должник собственность обязательство владение римский произведение исполнение юридический требование
sbj5 :  материал структура молекула свойство углерод применение поверхность полимер кристалл металл электронный твердый трехмерный алмаз углеродный
sbj6 :  книга фильм автор кино написать документ жанр театр тема кинематограф академия герой анекдот екатерина культовый
sbj7 :  смерть ритуал террор пра

In [8]:
batch_vectorizer = artm.BatchVectorizer(data_path="lectures.txt", 
                                        data_format="vowpal_wabbit", 
                                        target_folder='batches', 
                                        batch_size=100)
T = 30   # количество тем
topic_names = ["sbj" + str(i) for i in range(T - 1)] + ["bcg"]
class_ids = {'text': 1.0, 'author': 1.0}
random.seed(1)
np.random.seed(1)
model = artm.ARTM(num_topics=T, topic_names=topic_names, num_processors=2, class_ids={'text':1, 'author':1},
                  reuse_theta=True, cache_theta=True)

# model = artm.ARTM(num_topics=T, topic_names=topic_names, class_ids=class_ids, seed=1)
dictionary = artm.Dictionary('PostnaukaDictionary')
dictionary.gather(data_path=batch_vectorizer.data_path)
model.initialize(dictionary=dictionary)
model.scores.add(artm.TopTokensScore(name="TextTopTokensScore", num_tokens=15, class_id="text"))
model.scores.add(artm.TopTokensScore(name="AuthorTopTokensScore", num_tokens=15, class_id="author"))
model.regularizers.add(artm.SmoothSparsePhiRegularizer(
    name='SmoothRegularizer', dictionary='PostnaukaDictionary', tau=1e5, 
    class_ids='text', topic_names=topic_names[-1]))
model.num_document_passes = 1
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=30)
model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparseRegularizer', 
    tau=-1e5, dictionary='PostnaukaDictionary', class_ids='text', 
    topic_names=topic_names[:-1]))
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)
def print_tokens(tokens):
    for topic, items in tokens.items():
        print(f'{topic}: {" ".join(items)}')
tokens = model.score_tracker["TextTopTokensScore"].last_tokens

print_tokens(tokens)
# print(model.score_tracker)
# print(tokens[-1])
# for topic_name in model.topic_names:
#     print(topic_name, ': ', tokens[topic_name])


sbj0: церковь власть русь средневековый император святой римский сага князь папа король александр византийский церковный рим
sbj1: планета земля атмосфера солнце солнечный белый газ карлик образовываться вокруг звезда спутник вода поверхность кислород
sbj2: система состояние трудность двигаться частота поведение искусственный точность относительность движение переход тест час самка самец
sbj3: культура понятие культурный восприятие сознание феномен личность ощущение воспринимать чувство переживание повседневный этика лосев глобальный
sbj4: право литература литературный искусство юрист должник собственность обязательство римский владение произведение писатель юридический собственник исполнение
sbj5: материал структура молекула свойство углерод применение полимер поверхность вещество кристалл металл электронный алмаз углеродный твердый
sbj6: фильм книга автор кино театр жанр кинематограф анекдот тема документ пьеса культовый академия герой екатерина
sbj7: смерть ритуал террор государь пр

In [13]:
batch_vectorizer = artm.BatchVectorizer(data_path='lectures.txt', data_format='vowpal_wabbit',
                                        target_folder='lectures_batches', batch_size=250)
T = 30   # количество тем
topic_names=["sbj"+str(i) for i in range(T-1)]+["bcg"]
model = artm.ARTM(num_topics=T, topic_names=topic_names, num_processors=2, class_ids={'text':1, 'author':1},
                  reuse_theta=True, cache_theta=True)
np.random.seed(1)
dictionary = artm.Dictionary('dict')
dictionary.gather(batch_vectorizer.data_path)
model.initialize(dictionary=dictionary)
model.scores.add(artm.TopTokensScore(name='top_tokens_score_mod1', class_id='text', num_tokens=15))
model.scores.add(artm.TopTokensScore(name='top_tokens_score_mod2', class_id='author', num_tokens=15))

model.regularizers.add(artm.SmoothSparsePhiRegularizer(tau=1e5, class_ids='text', dictionary='dict', topic_names='bcg'))
model.num_document_passes = 1
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=30)

# model.regularizers.add(artm.SmoothSparsePhiRegularizer(
#     name='SmoothRegularizer', dictionary='dict', tau=1e5, 
#     class_ids='text', topic_names=topic_names[-1]))
# model.num_document_passes = 1
# model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=30)

topic_names_cleared = list(topic_names).remove('bcg')
model.regularizers.add(artm.SmoothSparsePhiRegularizer(#name='SparseRegularizer', 
    tau=-1e5, class_ids='text', dictionary='dict', 
    topic_names=topic_names_cleared))
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)

# topic_names_cleared = list(topic_names).remove('bcg')
# model.regularizers.add(artm.SmoothSparsePhiRegularizer(tau=-1e5, class_ids='text', dictionary='dict',
#                                                        topic_names=topic_names_cleared))
# model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)

def print_tokens(tokens):
    for topic, items in tokens.items():
        print(f'{topic}: {" ".join(items)}')
tokens = model.score_tracker["top_tokens_score_mod1"].last_tokens

print_tokens(tokens)



sbj0: церковь власть средневековый император русь святой римский папа князь сага король александр византийский вера рим
sbj1: планета земля атмосфера солнце солнечный белый газ образовываться карлик вокруг вода поверхность спутник орбита океан
sbj2: система трудность поведение частота двигаться точность искусственный движение относительность тест час самка задание переход часы
sbj3: культура понятие культурный восприятие сознание феномен личность воспринимать ощущение чувство повседневный переживание детство этика индивид
sbj4: право литература искусство литературный юрист писатель должник собственность обязательство владение римский произведение исполнение юридический требование
sbj5: материал структура молекула свойство углерод применение поверхность полимер кристалл металл электронный твердый трехмерный алмаз углеродный
sbj6: книга фильм автор кино написать документ жанр театр тема кинематограф академия герой анекдот екатерина культовый
sbj7: смерть ритуал террор правитель государь 

In [18]:
topic_names[:-1]

['sbj0',
 'sbj1',
 'sbj2',
 'sbj3',
 'sbj4',
 'sbj5',
 'sbj6',
 'sbj7',
 'sbj8',
 'sbj9',
 'sbj10',
 'sbj11',
 'sbj12',
 'sbj13',
 'sbj14',
 'sbj15',
 'sbj16',
 'sbj17',
 'sbj18',
 'sbj19',
 'sbj20',
 'sbj21',
 'sbj22',
 'sbj23',
 'sbj24',
 'sbj25',
 'sbj26',
 'sbj27',
 'sbj28']